In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# Step 1: Read all the Excel files over the 7 days into pandas dataframe
file_dates = ['19.04.2021', '20.04.2021', '21.04.2021', '22.04.2021', '23.04.2021', '24.04.2021', '25.04.2021']
dfs = []

for date in file_dates:
    file_path = f'VP_{date}.xlsx'
    df = pd.read_excel(file_path)
    dfs.append(df)

In [3]:
# Step 2: Concatenate the dataframes into a single dataframe
combined_df_webshops = pd.concat(dfs)
combined_df_webshops.drop_duplicates(subset=["Artikel Shop-Nummer"], keep='first') # drop the duplicated Artikel Shop-Nummer

,Shop ID,Artikel Shop-Nummer,Verkaufspreis,Datum
0,1,33,22.70,2021-04-19
1,1,119,17.90,2021-04-19
2,1,120,17.90,2021-04-19
3,1,121,17.90,2021-04-19
4,1,122,19.90,2021-04-19
...,...,...,...,...
290551,1,2747255,38.95,2021-04-25
290552,1,2747256,38.95,2021-04-25
290553,1,2747257,38.95,2021-04-25
290554,1,2747258,38.95,2021-04-25


In [4]:
# Step 3: Rename the column name of the combined file, to prepare for the merge
combined_df_webshops.rename(columns = {'Artikel Shop-Nummer':'Artikelnummer'}, inplace = True) 
print(combined_df_webshops)

        Shop ID  Artikelnummer  Verkaufspreis      Datum
0             1             33          22.70 2021-04-19
1             1            119          17.90 2021-04-19
2             1            120          17.90 2021-04-19
3             1            121          17.90 2021-04-19
4             1            122          19.90 2021-04-19
...         ...            ...            ...        ...
409020        3        2746277          39.95 2021-04-25
409021        3        2746278          34.95 2021-04-25
409022        3        2746279         119.90 2021-04-25
409023        3        2746580          34.95 2021-04-25
409024        3        2747223         201.95 2021-04-25

[2750849 rows x 4 columns]


In [5]:
# Step 4: Merge own webshop data with competitor data based on product number to indicate the product category
competitors_data = pd.read_csv("Konkurrenzdaten_new.csv")

merged_data = pd.merge(combined_df_webshops, competitors_data[['Artikelnummer', 'Artikelkategorie']], on='Artikelnummer', how='left')

print(merged_data)

          Shop ID  Artikelnummer  Verkaufspreis      Datum  \
0               1             33          22.70 2021-04-19   
1               1             33          22.70 2021-04-19   
2               1             33          22.70 2021-04-19   
3               1             33          22.70 2021-04-19   
4               1             33          22.70 2021-04-19   
...           ...            ...            ...        ...   
53559895        3        2746580          34.95 2021-04-25   
53559896        3        2747223         201.95 2021-04-25   
53559897        3        2747223         201.95 2021-04-25   
53559898        3        2747223         201.95 2021-04-25   
53559899        3        2747223         201.95 2021-04-25   

              Artikelkategorie  
0            Kunst & Literatur  
1            Kunst & Literatur  
2            Kunst & Literatur  
3            Kunst & Literatur  
4            Kunst & Literatur  
...                        ...  
53559895  Mobiltelefon Z

In [6]:
## Drop the columns I don't really need
merged_data.drop(['Verkaufspreis', 'Datum'], axis=1, inplace=True)

print(merged_data)

          Shop ID  Artikelnummer      Artikelkategorie
0               1             33     Kunst & Literatur
1               1             33     Kunst & Literatur
2               1             33     Kunst & Literatur
3               1             33     Kunst & Literatur
4               1             33     Kunst & Literatur
...           ...            ...                   ...
53559895        3        2746580  Mobiltelefon Zubehör
53559896        3        2747223       Aktenvernichter
53559897        3        2747223       Aktenvernichter
53559898        3        2747223       Aktenvernichter
53559899        3        2747223       Aktenvernichter

[53559900 rows x 3 columns]


In [7]:
# Step 5: Identify unique product categories
unique_own_categories = merged_data['Artikelkategorie'].unique()
unique_competitor_categories = competitors_data['Artikelkategorie'].unique()

In [8]:
# Step 6: Compare product category differences
own_only_categories = set(unique_own_categories) - set(unique_competitor_categories)
competitor_only_categories = set(unique_competitor_categories) - set(unique_own_categories)
overlap_categories = set(unique_own_categories) & set(unique_competitor_categories)

own_only_categories_number = len(own_only_categories)
competitor_only_categories_number = len(competitor_only_categories)
overlap_categories_number = len(overlap_categories)


# To make sure if the set is "nan", the number of the category should be "0"

# Check if the set contains only "nan"
if len(own_only_categories) == 1 and math.isnan(next(iter(own_only_categories))):
    # If the set contains only nan, assign 0 to the length of this set
    own_only_categories_number = 0
else:
    # If the set contains other values or is empty, use its normal length
    own_only_categories_number = len(own_only_categories)
print(f"Unique categories in your own webshops only: {own_only_categories_number} category/categories\n\n")
    

# Same to chect another set competitor_only_categories
if len(competitor_only_categories) == 1 and math.isnan(next(iter(competitor_only_categories))): 
    competitor_only_categories_number = 0
else:
    competitor_only_categories_number = len(competitor_only_categories)
print(f"Unique categories in competitors: {competitor_only_categories_number} category/categories\n\n")
    

# Same to chect another set overlap_categories
if len(overlap_categories) == 1 and math.isnan(next(iter(overlap_categories))):
    # If the set contains only nan, assign 0 to the length of this set
    overlap_categories_number = 0
else:
    # If the set contains other values or is empty, use its normal length
    overlap_categories_number = len(overlap_categories)
print(f"Categories common to both in total: {overlap_categories_number} category/categories\n\n")


print(f"Unique categories in your own webshops only: {own_only_categories}\n\n")
print(f"Categories common to both: {overlap_categories}\n\n")
print(f"Unique categories in competitors: {competitor_only_categories}\n\n")

Unique categories in your own webshops only: 0 category/categories


Unique categories in competitors: 2 category/categories


Categories common to both in total: 1583 category/categories


Unique categories in your own webshops only: {nan}


Categories common to both: {'Kompaktkameras', 'Versandboxen', 'Backblech', 'Spielfiguren Zubehör', 'Topflappen & Ofenhandschuhe', 'Netzwerkkameras', 'IPL Haarentfernung', 'Kopfkissen', 'Modelliermasse', 'Grafiktablets', 'Rettungsmittel', 'Lasermessgeräte', 'Kinderstühle & Sessel', 'Lippenstift & Lipgloss', 'Greifer & Haken Zubehör', 'Mobiltelefon Akku', 'Objektivzubehör', 'Baby Pflegeset', 'Milchpumpe Zubehör', 'Hot Stone Platten', 'Rasierer- & Haarentfernungszubehör', 'Gewächshäuser & Pflanzenschutz', 'Schwimmflossen', 'Medienkonverter', 'Vogel Spielzeug', 'Haarcoloration', 'Katzenbedarf', 'Baby Duschgel & Badezusätze', 'Nagelöl-/Crème', 'Audiorecording', 'Poolpumpen', 'Etiketten- & Belegdrucker', 'Pastamaschinen', 'Knöpfe', 'Fahrzeughalterungen'